In [1]:
import psycopg as pg
import pandas as pd
import json
from typing import Any

In [39]:
def get_login_str(file_p:str) -> str:
	with open(file_p) as f:
		data = json.load(f)
	login=data['pg']
	login_str = 'dbname={dbname} user={user} host={host} port={port} connect_timeout=10 password={password}'.format(
    dbname=login['dbname'],user=login['user'],host=login['host'],port=login['port'],password=login['password']
	)
	return login_str


def db_operate(sql_str:str) -> list[dict[str, int|str] | Any]:
  loggin_str: str = get_login_str(file_p = '.access.json')
  with pg.connect(loggin_str) as conn:
    with conn.cursor() as cur:
      ## 以查询数据库中计算名称、单点能、零点能、udft单点能、udft零点能、tdft的T1为例，如需查询其他数据请使用postgres的sql语法修改拼接字串
      query_all= f"SELECT name_calc, energy, corrections->>'zeroPoint', udft->>'energy',  udft->'corrections'->>'zeroPoint', tdft->'s0_t1'->>'energy' FROM moldata.main WHERE name_calc IN {sql_str}"
      cur.execute(query_all) # type: ignore
      all=[{
        'name_calc':i[0] if i[0] else 'null',
        'udft_t1':((float(i[3]) + float(i[4])) - (float(i[1]) + float(i[2]))) * 27.2114  if (i[1] and i[2] and i[3] and i[4]) else 'null',
        'tdft_t1':float(i[5][1:-1]) if i[5] else 'null',
      } for i in cur.fetchall()]
      ## print(all[:2])
      return all  # type: ignore

In [34]:
raw_name_str='''220411-EBL-0020
191126-EBL-0031
211214-EBL-0020
220906-EBL-0024
220916-EBL-0024
220803-EBL-0010
221011-EBH-0003
220902-EBL-0011
221212-EBL-0016
230112-EBL-0010
230110-EBL-0017
220908-EBL-0046
191218-EBL-0020
230113-EBL-0010
230113-EBL-0017
201229-EBL-0025
230426-EBL-0061
230515-EBL-0003
230602-EBL-0020
230904-EBL-0092
230112-EBL-0009
230714-EBL-0045
220822-EBL-0002
191218-EBL-0019
210122-EBL-0026
230426-EBL-0064
230519-EBL-0040
230524-EBL-0013
210205-EBL-0230
220830-EBL-0023
220906-EBL-0032
220428-EBL-0085
220420-EBL-0043
210205-EBL-0237
201125-EBL-0035
220627-EBL-0102
220907-EBL-0067
200403-EBL-0031
210201-EBL-0019
221019-EBL-0031
220830-HTL-0004
220914-EBL-0011
220902-EBL-0017
200413-EBL-0005
220919-EBL-0118
210513-EBL-0022
221014-EBL-0082
230217-EBL-0068
220211-EBL-0125
210927-EBL-0093
221025-EBL-0040
230421-EBL-0052
220610-EBL-0001
230324-EBL-0174
230324-EBL-0187
210929-EBL-0376
210205-EBL-0032
210121-EBL-0044
'''

In [35]:
# 拼接目标字串
name_list = raw_name_str.split('\n')
name_str = "('" + "','".join(name_list) + "')"
name_str

"('220411-EBL-0020','191126-EBL-0031','211214-EBL-0020','220906-EBL-0024','220916-EBL-0024','220803-EBL-0010','221011-EBH-0003','220902-EBL-0011','221212-EBL-0016','230112-EBL-0010','230110-EBL-0017','220908-EBL-0046','191218-EBL-0020','230113-EBL-0010','230113-EBL-0017','201229-EBL-0025','230426-EBL-0061','230515-EBL-0003','230602-EBL-0020','230904-EBL-0092','230112-EBL-0009','230714-EBL-0045','220822-EBL-0002','191218-EBL-0019','210122-EBL-0026','230426-EBL-0064','230519-EBL-0040','230524-EBL-0013','210205-EBL-0230','220830-EBL-0023','220906-EBL-0032','220428-EBL-0085','220420-EBL-0043','210205-EBL-0237','201125-EBL-0035','220627-EBL-0102','220907-EBL-0067','200403-EBL-0031','210201-EBL-0019','221019-EBL-0031','220830-HTL-0004','220914-EBL-0011','220902-EBL-0017','200413-EBL-0005','220919-EBL-0118','210513-EBL-0022','221014-EBL-0082','230217-EBL-0068','220211-EBL-0125','210927-EBL-0093','221025-EBL-0040','230421-EBL-0052','220610-EBL-0001','230324-EBL-0174','230324-EBL-0187','210929-

In [41]:
# 执行查询
data=db_operate(name_str)
data=[[i['name_calc'], i['udft_t1'], i['tdft_t1']] for i in data]

In [42]:
# 转换为pandas数据结构，然后保存为csv文件
pd_data = pd.DataFrame(data,columns=['name','udft_t1','tdft-t1'])
pd_data.to_csv('data.csv')